In [1]:
import numpy as np
%matplotlib inline
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor

In [ ]:
class TrijetProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "")
        Jet_axis = hist.Bin("Jet_pt", "Jet [GeV]", 50, 0, 125)
        b-tag_axis = hist.Bin("b-tag", "b-tagging discriminant")
        
        self._accumulator = processor.dict_accumulator({
            'Jet_pt': hist.Hist("Counts", dataset_axis, Jet_axis),
            'b-tag': hist.Hist("Counts", dataset_axis, b-tag_axis)
            'cutflow': processor.defaultdict_accumulator(int)
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()
        
        dataset = df["dataset"]
        
        jets = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt=df['Jet_pt'].content,
            eta=df['Jet_eta'].content,
            phi=df['Jet_phi'].content,
            mass=df['Jet_mass'].content,
            b_tag=df['Jet_btagCSVV2']
            )
        
        output['Jet_pt'].fill(dataset=dataset, Jet_pt=)
        output['b-tag'].fill(dataset=dataset, b-tag=)
        return output

    def postprocess(self, accumulator):
        return accumulator

In [ ]:
fileset = {'Trijets': ["../data/C9B654CF-0B51-4444-A177-479200DCC429.root"]}
output = processor.run_uproot_job(fileset,
                                 treename='Events',
                                 processor_instance=TrijetProcessor(),
                                 executor=processor.futures_executor,
                                 executor_args={'workers':4},
                                 chunksize = 500000)